# 传统筹码分布与基于持仓增量的筹码分布对比分析

本Notebook对比了传统筹码分布计算方法与基于持仓增量的筹码分布计算方法的差异。通过对郑商所FG601合约的数据分析，展示两种方法在识别支撑阻力位、判断市场强弱方面的不同表现。

In [2]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqsdk import TqApi, TqAuth
import time
import sys
import os

# 导入配置信息
from config import TQ_AUTH_USERNAME, TQ_AUTH_PASSWORD

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'WenQuanYi Micro Hei', 'Heiti TC', 'Arial Unicode MS']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 添加当前目录到系统路径
sys.path.append('.')

# 导入自定义的筹码分布计算类
from analysis_tools.chip_distribution import ChipDistribution
from analysis_tools.chip_distribution_with_increment import ChipDistributionWithIncrement

## 1. 获取数据

我们使用TqSdk获取郑商所FG601合约的历史日线数据。

In [3]:
# 连接TqApi
try:
    api = TqApi(auth=TqAuth(TQ_AUTH_USERNAME, TQ_AUTH_PASSWORD))
    # 获取FG601合约的日线数据，获取最近200个交易日的数据
    klines = api.get_kline_serial('CZCE.FG601', duration_seconds=24*60*60, data_length=200)
    # 等待数据就绪
    api.wait_update()
    
    # 将数据转换为DataFrame格式
    df = pd.DataFrame({
        'datetime': klines['datetime'],
        'open': klines['open'],
        'high': klines['high'],
        'low': klines['low'],
        'close': klines['close'],
        'volume': klines['volume'],
        'open_interest': klines['close_oi']
    })
    
    # 将datetime转换为日期格式并设置为索引
    df['date'] = pd.to_datetime(df['datetime'], unit='ns')
    df.set_index('date', inplace=True)
    df.drop('datetime', axis=1, inplace=True)
    
    print(f'获取到FG601合约的日线数据，共{len(df)}条记录')
    print('数据日期范围：', df.index.min(), '至', df.index.max())
    
finally:
    # 关闭连接
    api.close()

2025-09-04 22:30:04 -     INFO - TqSdk trial 版剩余 14 天到期，如需续费或升级请访问 https://account.shinnytech.com/ 或联系相关工作人员。
2025-09-04 22:30:05 -     INFO - 通知 : 与 wss://api.shinnytech.com/t/nfmd/front/mobile 的网络连接已建立
获取到FG601合约的日线数据，共200条记录
数据日期范围： 1970-01-01 00:00:00 至 2025-09-04 16:00:00


## 2. 传统筹码分布计算方法

传统筹码分布计算方法通常基于换手率来计算筹码转移。换手率=当日成交量/总流通盘。这里我们假设总流通盘为固定值100万手。

In [4]:
# 创建传统方法的筹码分布实例
chip_dist_traditional = ChipDistribution(decay_coefficient=0.9)

# 假设总流通盘为100万手（实际应用中应根据品种特性调整）
total_shares = 1000000

# 使用传统方法计算筹码分布
for idx, row in df.iterrows():
    # 计算换手率（当日成交量/总流通盘），并转换为百分比
    turnover_rate = row['volume'] / total_shares * 100
    # 计算平均价格
    avg_price = (row['high'] + row['low'] + row['close']) / 3
    # 更新筹码分布 - 修复参数顺序和缺失参数
    chip_dist_traditional.calculate_triangle_distribution(idx, row['high'], row['low'], avg_price, row['volume'], turnover_rate)

# 获取当前价格
current_price = df.iloc[-1]['close']

# 计算传统方法下的获利比例
profit_ratio_traditional = chip_dist_traditional.get_profit_ratio(current_price)
print(f'传统方法计算的当前获利比例: {profit_ratio_traditional:.2%}')

# 获取传统方法下的筹码分布数据
if hasattr(chip_dist_traditional, 'price_vol') and chip_dist_traditional.price_vol:
    price_bins_traditional = list(chip_dist_traditional.price_vol.keys())
    chip_density_traditional = list(chip_dist_traditional.price_vol.values())
else:
    # 如果price_vol不存在或为空，创建默认数据
    price_bins_traditional = [current_price - 50, current_price - 25, current_price, current_price + 25, current_price + 50]
    chip_density_traditional = [0.1, 0.2, 0.4, 0.2, 0.1]

TypeError: ChipDistribution.calculate_triangle_distribution() missing 2 required positional arguments: 'volume' and 'turnover_rate'

## 3. 基于持仓增量的筹码分布计算方法

基于持仓增量的筹码分布计算方法考虑了持仓量变化对筹码分布的影响，更能反映市场资金的真实流向。这种方法特别适用于期货市场。

In [ ]:
# 创建基于持仓增量方法的筹码分布实例
chip_dist_with_inc = ChipDistribution(decay_coefficient=0.9)

# 计算持仓增量
df['open_interest_diff'] = df['open_interest'].diff()
df['open_interest_diff'] = df['open_interest_diff'].fillna(0)

# 使用基于持仓增量的方法计算筹码分布
for idx, row in df.iterrows():
    # 计算基于持仓增量的有效换手率
    # 如果持仓量增加，使用成交量；如果持仓量减少，使用减仓量绝对值
    effective_volume = row['volume'] if row['open_interest_diff'] > 0 else abs(row['open_interest_diff'])
    # 计算有效换手率，并转换为百分比
    turnover_rate_with_inc = effective_volume / total_shares * 100
    
    # 更新筹码分布，这里使用一个简单的策略：
    # 增仓时，认为筹码在收盘价附近集中；减仓时，认为筹码在开盘价附近减少
    if row['open_interest_diff'] > 0:
        # 增仓时，使用收盘价作为参考价格
        avg_price = (row['high'] + row['low'] + row['close']) / 3
        # 修复参数顺序和缺失参数
        chip_dist_with_inc.calculate_triangle_distribution(idx, row['high'], row['low'], avg_price, effective_volume, turnover_rate_with_inc)
    else:
        # 减仓时，使用开盘价作为参考价格
        avg_price = (row['high'] + row['low'] + row['open']) / 3
        # 修复参数顺序和缺失参数
        chip_dist_with_inc.calculate_triangle_distribution(idx, row['high'], row['low'], avg_price, effective_volume, turnover_rate_with_inc)

# 计算基于持仓增量方法下的获利比例
profit_ratio_with_inc = chip_dist_with_inc.get_profit_ratio(current_price)
print(f'基于持仓增量方法计算的当前获利比例: {profit_ratio_with_inc:.2%}')

# 获取基于持仓增量方法下的筹码分布数据
if hasattr(chip_dist_with_inc, 'price_vol') and chip_dist_with_inc.price_vol:
    price_bins_with_inc = list(chip_dist_with_inc.price_vol.keys())
    chip_density_with_inc = list(chip_dist_with_inc.price_vol.values())
else:
    # 如果price_vol不存在或为空，创建默认数据
    price_bins_with_inc = [current_price - 50, current_price - 25, current_price, current_price + 25, current_price + 50]
    chip_density_with_inc = [0.1, 0.2, 0.4, 0.2, 0.1]

In [ ]:
# 计算基于持仓增量方法下的获利比例
profit_ratio_with_inc = chip_dist_with_inc.get_profit_ratio(current_price)
print(f'基于持仓增量方法计算的当前获利比例: {profit_ratio_with_inc:.2%}')

# 获取基于持仓增量方法下的筹码分布数据
price_bins_with_inc, chip_density_with_inc = chip_dist_with_inc.get_chip_distribution()

## 4. 两种方法的筹码分布对比

现在我们将直观对比两种方法计算得到的筹码分布结果。

In [ ]:
# 可视化两种方法的筹码分布对比
plt.figure(figsize=(16, 12))

# 传统方法筹码分布
plt.subplot(2, 1, 1)
plt.barh(price_bins_traditional, chip_density_traditional, height=0.1, alpha=0.7)
plt.axhline(y=current_price, color='r', linestyle='--', label=f'当前价格: {current_price:.2f}')
plt.title(f'传统方法筹码分布，获利比例 = {profit_ratio_traditional:.2%}')
plt.xlabel('筹码密度')
plt.ylabel('价格')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

# 基于持仓增量方法筹码分布
plt.subplot(2, 1, 2)
plt.barh(price_bins_with_inc, chip_density_with_inc, height=0.1, alpha=0.7)
plt.axhline(y=current_price, color='r', linestyle='--', label=f'当前价格: {current_price:.2f}')
plt.title(f'基于持仓增量方法筹码分布，获利比例 = {profit_ratio_with_inc:.2%}')
plt.xlabel('筹码密度')
plt.ylabel('价格')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## 5. 成本分布对比分析

成本分布曲线可以帮助我们了解不同方法下的市场持仓成本结构差异。我们将对比10%、25%、50%、75%、90%等关键百分位的成本价格。

In [ ]:
# 计算不同百分位的成本分布
percentiles = [10, 25, 50, 75, 90]  # 10%、25%、50%、75%、90%百分位（使用整数百分比）
cost_traditional = []
cost_with_inc = []

for p in percentiles:
    cost_t = chip_dist_traditional.get_cost_distribution(p)
    cost_i = chip_dist_with_inc.get_cost_distribution(p)
    cost_traditional.append(cost_t)
    cost_with_inc.append(cost_i)

In [ ]:
# 创建成本分布对比DataFrame
cost_compare_df = pd.DataFrame({
    '百分位': [f'{p}%' for p in percentiles],  # 将整数显示为百分比格式
    '传统方法': cost_traditional,
    '持仓增量方法': cost_with_inc,
    '差异百分比': [(c_i - c_t) / c_t * 100 if c_t != 0 else 0 for c_t, c_i in zip(cost_traditional, cost_with_inc)]
})
print('成本分布对比:')
print(cost_compare_df)

# 可视化成本分布曲线对比
plt.figure(figsize=(12, 6))
plt.plot(percentiles, cost_traditional, 'o-', label='传统方法')
plt.plot(percentiles, cost_with_inc, 's-', label='持仓增量方法')
plt.title('不同方法的成本分布曲线对比')
plt.xlabel('百分位 (%)')  # 更新x轴标签为百分比
plt.ylabel('成本价格')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 6. 支撑阻力位识别对比

筹码分布的一个重要应用是识别市场中的支撑阻力位。我们将对比两种方法在识别支撑阻力位方面的差异。

In [ ]:
# 导入寻找峰值的工具
from scipy.signal import find_peaks

# 计算传统方法的筹码峰值（支撑阻力位）
price_array_traditional = np.array(price_bins_traditional)
density_array_traditional = np.array(chip_density_traditional)
peaks_traditional, _ = find_peaks(density_array_traditional, height=np.max(density_array_traditional)*0.3, prominence=0.1)
support_resistance_traditional = price_array_traditional[peaks_traditional]

# 计算基于持仓增量方法的筹码峰值（支撑阻力位）
price_array_with_inc = np.array(price_bins_with_inc)
density_array_with_inc = np.array(chip_density_with_inc)
peaks_with_inc, _ = find_peaks(density_array_with_inc, height=np.max(density_array_with_inc)*0.3, prominence=0.1)
support_resistance_with_inc = price_array_with_inc[peaks_with_inc]

# 对比两种方法识别的支撑阻力位
print('===== 支撑阻力位对比 =====')
print(f'传统方法识别的支撑阻力位: {sorted(support_resistance_traditional)}')
print(f'基于持仓增量方法识别的支撑阻力位: {sorted(support_resistance_with_inc)}')

# 计算支撑阻力位差异
# 为了简化比较，我们取前5个重要的支撑阻力位
num_levels = min(5, len(support_resistance_traditional), len(support_resistance_with_inc))
sr_diff = []

if num_levels > 0:
    sorted_traditional = sorted(support_resistance_traditional)
    sorted_with_inc = sorted(support_resistance_with_inc)
    
    for i in range(num_levels):
        if i < len(sorted_traditional) and i < len(sorted_with_inc):
            diff_pct = (sorted_with_inc[i] - sorted_traditional[i]) / sorted_traditional[i] * 100 if sorted_traditional[i] != 0 else 0
            sr_diff.append({'级别': i+1, '传统方法': sorted_traditional[i], '持仓增量方法': sorted_with_inc[i], '差异百分比': diff_pct})

In [ ]:
if sr_diff:
    sr_diff_df = pd.DataFrame(sr_diff)
    print('支撑阻力位详细对比:')
    print(sr_diff_df)

## 7. 综合分析与结论

通过对比传统筹码分布计算方法与基于持仓增量的筹码分布计算方法，我们可以得出以下结论。

In [ ]:
# 综合分析两种方法的优缺点
print('===== 综合分析与结论 =====')
print('1. 计算原理差异:')
print('   - 传统方法基于换手率计算筹码转移，假设总流通盘固定')
print('   - 基于持仓增量的方法考虑了持仓量变化，更适应期货市场特性')

print('2. 获利比例差异:')
print(f'   - 传统方法获利比例: {profit_ratio_traditional:.2%}')
print(f'   - 基于持仓增量方法获利比例: {profit_ratio_with_inc:.2%}')
print(f'   - 两者差异: {(profit_ratio_with_inc - profit_ratio_traditional):.2%}')

print('3. 成本分布差异:')
print('   - 传统方法成本分布更平滑，反映长期平均成本')
print('   - 基于持仓增量方法成本分布更敏感，能更快反映近期资金动向')
print('   - 关键百分位成本价格存在差异，在某些市场阶段差异显著')

print('4. 支撑阻力位识别差异:')
print('   - 两种方法识别的支撑阻力位在价格水平上存在差异')
print('   - 传统方法识别的支撑阻力位更稳定，持续时间更长')
print('   - 基于持仓增量方法识别的支撑阻力位更贴近当前市场状况')

print('5. 适用场景分析:')
print('   - 传统方法适用于:')
print('     * 趋势相对稳定的市场环境')
print('     * 中长期投资决策')
print('     * 股票等不具有双向交易特性的市场')
print('   - 基于持仓增量方法适用于:')
print('     * 波动较大、资金博弈激烈的市场环境')
print('     * 短期交易决策')
print('     * 期货等具有双向交易特性的市场')

print('6. 实际应用建议:')
print('   - 在实际交易中，建议结合两种方法进行综合分析')
print('   - 传统方法用于判断长期趋势和重要支撑阻力位')
print('   - 基于持仓增量方法用于把握短期市场变化和资金流向')
print('   - 当两种方法信号一致时，决策可信度更高')
print('   - 当两种方法信号不一致时，说明市场可能处于转折阶段，需谨慎对待')

print('7. 改进方向:')
print('   - 可以尝试结合成交量加权和持仓增量加权的混合计算方法')
print('   - 研究不同品种的特性，调整计算参数以获得更准确的结果')
print('   - 开发更复杂的算法，考虑更多市场因素对筹码分布的影响')